In [84]:
import pandas as pd
import numpy as np

In [10]:
claims_db_matched = pd.read_csv('friday_sprint/gimme_pricing/claims_weight_db_matched_weights_v0.6.8.csv')

#mapping from claim to zip
zip_mapping_df = claims_db_matched.drop_duplicates(subset=['claim_id'])[['claim_id', 'zip']]
zip_mapping_df['claim_id'] = zip_mapping_df['claim_id'].astype('int')
zip_mapping_df['zip'] = zip_mapping_df['zip'].astype('int')

claim_id_weight_comparison = pd.read_excel('friday_sprint/gimme_pricing/claim_id_weight_comparison_v0.6.8_pickup4.xlsx')

gimme_pricing = pd.read_csv('friday_sprint/gimme_pricing/20220127-1856_container prices per zip.csv', index_col='Unnamed: 0')

In [44]:
app_covered_zip = list(gimme_pricing.zip_code.unique())
zip_mapping_df['zip_app_covered'] = zip_mapping_df['zip'].isin(app_covered_zip)

In [45]:
zip_mapping_df

,claim_id,zip,zip_app_covered
0,1,78003,True
34,2,79901,True
41,3,75043,True
48,4,78942,True
58,5,78013,True
...,...,...,...
85092,11817,76226,True
85094,11818,75028,True
85103,11819,76092,True
85110,11820,76210,True


In [47]:
claim_id_weight_comparison = claim_id_weight_comparison.merge(zip_mapping_df, on='claim_id')

In [48]:
claim_id_weight_comparison.head()

,claim_id,total_claims,total_items,total_truck_weight,total_truck_cost,ID_matched_fraction,matched_fraction,weight_estimation_ustons_max_max,weight_estimation_ustons_mean_mean,weight_estimation_ustons_median_median,weight_estimation_ustons_min_min,weight_estimation_ustons_max_zero,weight_estimation_ustons_mean_zero,weight_estimation_ustons_median_zero,weight_estimation_ustons_min_zero,excessive_truck_weight_median_median,relative_excessive_truck_weight_median_median,zip,zip_app_covered
0,1,31,37,45.5,10943.0,0.84,0.97,109.244434,41.809806,35.011553,14.085450,36.964296,20.879689,18.078093,10.391235,10.488447,23.051531,78003,True
1,2,7,10,NaN,NaN,0.71,1.00,3.425469,3.132334,3.122454,2.869439,3.215469,3.032454,3.032454,2.849439,NaN,NaN,79901,True
2,3,6,7,4.0,98.0,1.00,1.00,6.527688,4.541365,4.542626,2.553782,6.527688,4.541365,4.542626,2.553782,-0.542626,-13.565642,75043,True
3,4,9,9,22.5,5793.0,0.78,1.00,3.973533,3.556503,3.533463,3.162513,3.973533,3.556503,3.533463,3.162513,18.966537,84.295719,78942,True
4,5,4,4,6.0,930.0,0.75,1.00,141.018972,22.012134,2.708985,2.403726,2.264972,2.264972,2.264972,2.264972,3.291015,54.850257,78013,True


In [109]:
result_list = []

for index,row in claim_id_weight_comparison.iterrows():
    
    #print progress
    if(index%100 == 0):
        print(index/len(claim_id_weight_comparison))
    
    #get pricing for zip code of claim
    zip_code = row['zip']
    weight = row['weight_estimation_ustons_mean_mean']
    relevant_prices = gimme_pricing[gimme_pricing['zip_code'] == zip_code].sort_values(by='price_per_collection')
    
    #set arbitrary price so that any would be cheaper
    cheapest_price = 999999999999999
    
    for pricing_index, pricing_row in relevant_prices.iterrows():
        
        needed_containers = weight / pricing_row['weight_limit_tons']
        needed_containers = np.ceil(needed_containers)
        gimme_price = needed_containers * float(pricing_row['price_per_collection'])
        
        #only store the price info if cheaper than all previous
        if gimme_price < cheapest_price:
            cheapest_price = gimme_price
            cheapest_dict = ({
                'claim_id' : row['claim_id'],
                'gimme_price': gimme_price,
                'provider': pricing_row['company_name'],
                'container_type': pricing_row['name'],
                'number_of_containers': needed_containers
            })
    
    result_list.append(cheapest_dict)
            
    #print(row['claim_id'])
    #print(cheapest_price)
    #print(cheapest_offer)
    #print('----------')

result_df = pd.DataFrame(result_list)

0.0
0.009405568096313018
0.018811136192626036
0.028216704288939052
0.03762227238525207
0.04702784048156509
0.056433408577878104
0.06583897667419113
0.07524454477050414
0.08465011286681716
0.09405568096313018
0.10346124905944319
0.11286681715575621
0.12227238525206922
0.13167795334838225
0.14108352144469527
0.1504890895410083
0.1598946576373213
0.16930022573363432
0.17870579382994733
0.18811136192626035
0.19751693002257337
0.20692249811888638
0.2163280662151994
0.22573363431151242
0.23513920240782543
0.24454477050413845
0.25395033860045146
0.2633559066967645
0.2727614747930775
0.28216704288939054
0.29157261098570353
0.3009781790820166
0.31038374717832956
0.3197893152746426
0.3291948833709556
0.33860045146726864
0.3480060195635816
0.35741158765989467
0.36681715575620766
0.3762227238525207
0.3856282919488337
0.39503386004514673
0.4044394281414597
0.41384499623777277
0.42325056433408575
0.4326561324303988
0.4420617005267118
0.45146726862302483
0.4608728367193379
0.47027840481565086
0.47968

In [146]:
output_df = claim_id_weight_comparison.merge(result_df, on="claim_id")
output_df['saving'] = output_df['total_truck_cost'] - output_df['gimme_price']
output_df = output_df[output_df.total_truck_cost.isna() == False]

In [147]:
output_df.to_excel('first_pricing_output.xlsx')

In [ ]:
#easiest version: all in one container
#then: surplus divided again
#then: distribute across all containers